In [1]:
import torch
import torch.nn as nn
import torch
import torch.optim as optim
from torch.utils.data import DataLoader
from tqdm import tqdm

from util import SST2Dataset, load_embedding_matrix

## A5.a

In [2]:
def collate_fn(batch):
    """
    Create a batch of data given a list of N sequences and labels. Sequences are stacked into a single tensor
    of shape (N, max_sequence_length), where max_sequence_length is the maximum length of any sequence in the
    batch. Sequences shorter than this length should be filled up with 0's. Also returns a tensor of shape (N, 1)
    containing the label of each sequence.

    :param batch: A list of size N, where each element is a tuple containing a sequence tensor and a single item
    tensor containing the true label of the sequence.

    :return: A tuple containing two tensors. The first tensor has shape (N, max_sequence_length) and contains all
    sequences. Sequences shorter than max_sequence_length are padded with 0s at the end. The second tensor
    has shape (N, 1) and contains all labels.
    """
    sentences, labels = zip(*batch)

    # torch.nn.utils.rnn.pad_sequence will find the max length in sentences and pad padding_value to it
    # set padding_value=0 as it's the token id for the padding token
    padded_sentences = torch.nn.utils.rnn.pad_sequence(sentences, batch_first=True, padding_value=0)
    
    # stack all labels into one tensor
    labels = torch.stack(labels)
    return padded_sentences, labels

## A5.b & A5.c

In [3]:
class RNNBinaryClassificationModel(nn.Module):
    def __init__(self, embedding_matrix, model_type):
        super().__init__()

        vocab_size = embedding_matrix.shape[0]
        embedding_dim = embedding_matrix.shape[1]

        # Construct embedding layer and initialize with given embedding matrix. Do not modify this code.
        self.embedding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim, padding_idx=0)
        self.embedding.weight.data = embedding_matrix

        hidden_size = 64
        if model_type == 'rnn':
            rnn = nn.RNN
        elif model_type == 'lstm':
            rnn = nn.LSTM
        elif model_type == 'gru':
            rnn = nn.GRU
        else:
            raise ValueError("model type must be in ('rnn', 'lstm', 'gru')")
            
        self.rnn = rnn(
            input_size=embedding_dim,
            hidden_size=hidden_size
        )
        self.fc = nn.Linear(hidden_size, 1) # output_dim = 1 because of binary classification
        self._loss = nn.CrossEntropyLoss()

    def forward(self, inputs):
        """
        Takes in a batch of data of shape (N, max_sequence_length). Returns a tensor of shape (N, 1), where each
        element corresponds to the prediction for the corresponding sequence.
        :param inputs: Tensor of shape (N, max_sequence_length) containing N sequences to make predictions for.
        :return: Tensor of predictions for each sequence of shape (N, 1).
        """
        
        embedding = self.embedding(inputs)
        rnn_output, hdn = self.rnn(embedding)
        logits = self.fc(rnn_output)
        return logits

    def loss(self, logits, targets):
        """
        Computes the binary cross-entropy loss.
        :param logits: Raw predictions from the model of shape (N, 1)
        :param targets: True labels of shape (N, 1)
        :return: Binary cross entropy loss between logits and targets as a scalar tensor.
        """
        return self._loss(logits, targets)

    def accuracy(self, logits, targets):
        """
        Computes the accuracy, i.e number of correct predictions / N.
        :param logits: Raw predictions from the model of shape (N, 1)
        :param targets: True labels of shape (N, 1)
        :return: Accuracy as a scalar tensor.
        """
        
        pred = torch.argmax(logits, axis=1)
        acc = (pred == targets).double().mean()
        return acc

## A5.d

In [4]:
# Training parameters
TRAINING_BATCH_SIZE = 32
NUM_EPOCHS = 16
LEARNING_RATE = 1e-3

# Batch size for validation, this only affects performance.
VAL_BATCH_SIZE = 128

In [5]:
def train(model_type):
    # Load datasets
    train_dataset = SST2Dataset("./SST-2/train.tsv")
    val_dataset = SST2Dataset("./SST-2/dev.tsv", train_dataset.vocab, train_dataset.reverse_vocab)

    # Create data loaders for creating and iterating over batches
    train_loader = DataLoader(train_dataset, batch_size=TRAINING_BATCH_SIZE, collate_fn=collate_fn, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=VAL_BATCH_SIZE, collate_fn=collate_fn)

    # Print out some random examples from the data
    print("Data examples:")
    random_indices = torch.randperm(len(train_dataset))[:8].tolist()
    for index in random_indices:
        sequence_indices, label = train_dataset.sentences[index], train_dataset.labels[index]
        sentiment = "Positive" if label == 1 else "Negative"
        sequence = train_dataset.indices_to_tokens(sequence_indices)
        print(f"Sentiment: {sentiment}. Sentence: {sequence}")
    print()

    embedding_matrix = load_embedding_matrix(train_dataset.vocab)

    model = RNNBinaryClassificationModel(embedding_matrix, model_type)
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

    for epoch in range(NUM_EPOCHS):
        # Total loss across train data
        train_loss = 0.
        # Total number of correctly predicted training labels
        train_correct = 0
        # Total number of training sequences processed
        train_seqs = 0

        tqdm_train_loader = tqdm(train_loader)
        print(f"Epoch {epoch + 1}/{NUM_EPOCHS}")

        model.train()
        for batch_idx, batch in enumerate(tqdm_train_loader):
            sentences_batch, labels_batch = batch

            # Make predictions
            logits = model(sentences_batch)

            # Compute loss and number of correct predictions
            loss = model.loss(logits, labels_batch)
            correct = model.accuracy(logits, labels_batch).item() * len(logits)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Accumulate metrics and update status
            train_loss += loss.item()
            train_correct += correct
            train_seqs += len(sentences_batch)
            tqdm_train_loader.set_description_str(
                f"[Loss]: {train_loss / (batch_idx + 1):.4f} [Acc]: {train_correct / train_seqs:.4f}")
        print()

        avg_train_loss = train_loss / len(tqdm_train_loader)
        train_accuracy = train_correct / train_seqs
        print(f"[Training Loss]: {avg_train_loss:.4f} [Training Accuracy]: {train_accuracy:.4f}")

        print("Validating")
        # Total loss across validation data
        val_loss = 0.
        # Total number of correctly predicted validation labels
        val_correct = 0
        # Total number of validation sequences processed
        val_seqs = 0

        tqdm_val_loader = tqdm(val_loader)

        model.eval()
        for batch_idx, batch in enumerate(tqdm_val_loader):
            sentences_batch, labels_batch = batch

            with torch.no_grad():
                # Make predictions
                logits = model(sentences_batch)

                # Compute loss and number of correct predictions and accumulate metrics and update status
                val_loss += model.loss(logits, labels_batch).item()
                val_correct += model.accuracy(logits, labels_batch).item() * len(logits)
                val_seqs += len(sentences_batch)
                tqdm_val_loader.set_description_str(
                    f"[Loss]: {val_loss / (batch_idx + 1):.4f} [Acc]: {val_correct / val_seqs:.4f}")
        print()

        avg_val_loss = val_loss / len(tqdm_val_loader)
        val_accuracy = val_correct / val_seqs
        print(f"[Validation Loss]: {avg_val_loss:.4f} [Validation Accuracy]: {val_accuracy:.4f}")

### A5.d RNN

In [6]:
train('rnn')

/mnt/efs/bruce.zhou/notebooks/misc/encoder-decoder/RNN/util.py:77: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  indexed_sentences = [torch.tensor(self.tokens_to_indices(sentence)) for sentence in sentences]


Data examples:
Sentiment: Positive. Sentence: an accomplished actress
Sentiment: Positive. Sentence: a successful example of the lovable-loser protagonist
Sentiment: Positive. Sentence: a vivid , sometimes surreal , glimpse into the mysteries of human behavior .
Sentiment: Negative. Sentence: walked out of runteldat
Sentiment: Negative. Sentence: relentlessly saccharine
Sentiment: Positive. Sentence: this charming but slight tale has warmth , wit and interesting characters compassionately portrayed .
Sentiment: Positive. Sentence: last year 's kubrick-meets-spielberg exercise
Sentiment: Negative. Sentence: substitute plot for personality



  0%|          | 0/2105 [00:00<?, ?it/s]/opt/conda/envs/cr_latest/lib/python3.6/site-packages/torch/autograd/__init__.py:132: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  allow_unreachable=True)  # allow_unreachable flag
[Loss]: 2.7732 [Acc]: 0.3109:   0%|          | 10/2105 [00:00<00:22, 92.21it/s]

Epoch 1/16


[Loss]: 1.0320 [Acc]: 0.4833:   0%|          | 10/2105 [00:00<00:20, 99.99it/s]

...


[Training Loss]: 0.8158 [Training Accuracy]: 0.6333
Validating

[Validation Loss]: 2.1827 [Validation Accuracy]: 0.1984
Epoch 15/16


[Loss]: 0.8778 [Acc]: 0.6317:   0%|          | 10/2105 [00:00<00:21, 98.68it/s]


[Training Loss]: 0.8013 [Training Accuracy]: 0.6452
Validating

[Validation Loss]: 2.0722 [Validation Accuracy]: 0.2477
Epoch 16/16


[Loss]: 2.2148 [Acc]: 0.1984: 100%|██████████| 7/7 [00:00<00:00, 162.86it/s]


[Training Loss]: 0.7813 [Training Accuracy]: 0.6514
Validating

[Validation Loss]: 2.2148 [Validation Accuracy]: 0.1984


### A5.d LSTM

In [7]:
train('lstm')

Data examples:
Sentiment: Positive. Sentence: for people who make movies and watch them
Sentiment: Positive. Sentence: a crisp psychological drama ( and ) a fascinating little thriller that would have been perfect for an old `` twilight zone '' episode .
Sentiment: Positive. Sentence: captures that perverse element of the kafkaesque where identity , overnight , is robbed and replaced with a persecuted `` other
Sentiment: Positive. Sentence: 's a very very strong `` b + . ''
Sentiment: Positive. Sentence: did n't hate this one
Sentiment: Positive. Sentence: australia is a weirdly beautiful place
Sentiment: Negative. Sentence: hymn and a cruel story of youth culture
Sentiment: Positive. Sentence: somehow manages to bring together kevin pollak , former wrestler chyna and dolly parton



[Loss]: 3.1969 [Acc]: 0.3693:   0%|          | 6/2105 [00:00<00:37, 56.26it/s]

Epoch 1/16


[Loss]: 1.1424 [Acc]: 0.4750:   0%|          | 0/2105 [00:00<?, ?it/s]


[Training Loss]: 1.1920 [Training Accuracy]: 0.4450
Validating

[Validation Loss]: 1.8124 [Validation Accuracy]: 0.3394
Epoch 2/16


...


[Training Loss]: 0.6390 [Training Accuracy]: 0.7145
Validating

[Validation Loss]: 1.6577 [Validation Accuracy]: 0.3761
Epoch 15/16


[Loss]: 0.5068 [Acc]: 0.8125:   0%|          | 0/2105 [00:00<?, ?it/s]


[Training Loss]: 0.6318 [Training Accuracy]: 0.7178
Validating

[Validation Loss]: 1.8529 [Validation Accuracy]: 0.3062
Epoch 16/16


[Loss]: 1.6981 [Acc]: 0.3578: 100%|██████████| 7/7 [00:00<00:00, 73.61it/s]


[Training Loss]: 0.6203 [Training Accuracy]: 0.7231
Validating

[Validation Loss]: 1.6981 [Validation Accuracy]: 0.3578


### A5.d GRU

In [8]:
train('gru')

Data examples:
Sentiment: Negative. Sentence: the film 's ending has a `` what was it all for
Sentiment: Positive. Sentence: smooth , shrewd , powerful act
Sentiment: Negative. Sentence: ( toback 's ) fondness for fancy split-screen , stuttering editing and pompous references to wittgenstein and kirkegaard ...
Sentiment: Positive. Sentence: a testament to de niro and director michael caton-jones
Sentiment: Negative. Sentence: die
Sentiment: Positive. Sentence: make us care about zelda 's ultimate fate
Sentiment: Negative. Sentence: you have left the theater
Sentiment: Negative. Sentence: feels impersonal , almost generic



[Loss]: 2.9010 [Acc]: 0.3608:   0%|          | 6/2105 [00:00<00:35, 59.17it/s]

Epoch 1/16


[Loss]: 1.0974 [Acc]: 0.4313:   0%|          | 0/2105 [00:00<?, ?it/s]


[Training Loss]: 1.1632 [Training Accuracy]: 0.4417
Validating

[Validation Loss]: 2.0271 [Validation Accuracy]: 0.3314
Epoch 2/16


...


[Training Loss]: 0.6518 [Training Accuracy]: 0.7114
Validating

[Validation Loss]: 1.8124 [Validation Accuracy]: 0.3452
Epoch 15/16


[Loss]: 0.6970 [Acc]: 0.7266:   0%|          | 0/2105 [00:00<?, ?it/s]


[Training Loss]: 0.6439 [Training Accuracy]: 0.7137
Validating

[Validation Loss]: 1.7607 [Validation Accuracy]: 0.3739
Epoch 16/16


[Loss]: 1.7518 [Acc]: 0.3876: 100%|██████████| 7/7 [00:00<00:00, 85.53it/s]


[Training Loss]: 0.6345 [Training Accuracy]: 0.7183
Validating

[Validation Loss]: 1.7518 [Validation Accuracy]: 0.3876


## A5.e

An RNN or LSTM have the advantage of "remembering" the past inputs, to improve performance over prediction of a time-series data. If you use a neural network over like the past 500 characters, this may work but the network just treat the data as a bunch of data without any specific indication of time. The network can learn the time representation only through gradient descent. RNN or LSTM however have "time" as a mechanism built into the model. The model loops through the model sequentially and have a real "sense of time" even before the model is trained. The model also have "memory" of previous data points to help the prediction. The architecture is based on the progress of time and the gradient are propagated through time as well. This is a much more intuitive way to process time-series data.

However in this problem of text classification, we don't have to assume such time-series structure of input. And the downside of such assumption only allows us use past information at a given word while we can definitely leverage the information from the entire sentence to make classfication decision.

## A5.f

The reason why it makes sense to feed the final hidden state of an RNN into the classification layer is because we can view the final hidden state of an RNN as an "embedding" of the input sequence, and hence it contains information about this input sequence and we can use this embedding in our classification layer. 

However, like mentioned in the above question, RNN only tends to remember information that are closer to the current state, and hence the information provided by the final hidden state might, to some extent, lose information from early tokens in the sentence. This can be considered as a sort of information vanish.

## A5.g

While RNN goes through each token, we can save the hidden state, and feed it to the classification layer, and compute the loss using the logit of current token tag prediction and its actual tag. Then when the RNN goes to the next token, we repeat the same procedure, i.e., get the hidden state, feed it to the classification layer, compute loss. We repeat this process until the end of the sentence. This way we'll have tags predicted for all tokens.

## A5.h

Funniest review:
    
makes a joke out of car chases for an hour and then gives us half an hour of car chases 